In [1]:
!pip install -r requirements.txt

  Found existing installation: Keras 2.0.8
    Uninstalling Keras-2.0.8:
      Successfully uninstalled Keras-2.0.8


the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [13]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_valid = np.array(h['valid'])
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])
        X_train, y_train = shuffle(X_train, y_train)
        y_train = one_hot_encode(y_train)
        X_valid, y_valid = shuffle(X_valid, y_valid)
        y_valid = one_hot_encode(y_valid)
        return X_train, y_train, X_valid, y_valid
    
from keras.models import *
from keras.layers import *

def make_model(input_shape):

    input_tensor = Input(input_shape)
    x = input_tensor
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    
#     x = Dense(10)(x)
#     x = Activation('softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     #model.compile(Adam(lr=1e-3), loss='categorical_crossentropy')
    
    return model
print("Done")

Done


VGG16

In [14]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=16, epochs=30, validation_data=(X_valid,y_valid))
model.save("models/vgg16-model.h5")
print("model save successed")

load data: bottleneck_VGG16.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 3s - loss: 5.7750 - acc: 0.3506 - val_loss: 4.1273 - val_acc: 0.2963

In [15]:
#test_premodels = ["bottleneck_ResNet50_test.h5", "bottleneck_Xception_test.h5", "bottleneck_InceptionV3_test.h5", 
#             "bottleneck_VGG16_test.h5", "bottleneck_VGG19_test.h5"]

test_premodel = "bottleneck_VGG16_test.h5"

X_test = []
with h5py.File("models/" + test_premodel, 'r') as h:
    X_test = np.array(h['test'])

assert(X_test.shape[1] == X_train.shape[1])

In [16]:
y_pred = model.predict(X_test, verbose=1)
#y_pred = y_pred.clip(min=0.005, max=0.995)
print()
print(y_pred[:10])

import pandas as pd
from keras.preprocessing.image import *

dir = "/ext/Data/distracted_driver_detection/"
model_image_size = 224

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(dir + "test/", (model_image_size, model_image_size), shuffle=False, 
                                         batch_size=16, class_mode=None)

l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    l.append( [name, *y_pred[i]] )
    #index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    #df.set_value(index-1, 'label', y_pred[i])
    #df.append(df2, ignore_index=True
l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df = df.sort_values(by='img')
df.to_csv('vgg16-pred.csv', index=None, float_format='%.3f')

78272/79726 [============================>.] - ETA: 0s 